In [1]:
import pandas as pd
import numpy as np
from bokeh.transform import linear_cmap
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Legend, CDSView, BooleanFilter
from bokeh.models import HoverTool
from bokeh.models import CustomJS, Div, Button
from bokeh import events
from bokeh.layouts import column, row
output_notebook()

Loading BokehJS ...

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [3]:
excel_source = pd.read_excel("Demand.xlsx",sheet_name=None)
units_pallet = excel_source.get("Units per Pallet")
breakout = excel_source.get("Breakout")
rates_umbra = excel_source.get("Umbra")
rates_thiele = excel_source.get("Thiele")
rates_parsons = excel_source.get("Parsons")
rates_ptech = excel_source.get("Premier Tech")
demand = excel_source.get("Demand")

excel_source2 = pd.read_excel("Output.xlsx",sheet_name=None)
out_cf = excel_source2.get("out_cf")
out_cf_orders = excel_source2.get("out_cf_orders")
out_fg = excel_source2.get("out_fg")
out_packline = excel_source2.get("out_packline")
out_bins = excel_source2.get("out_bins")

units_pallet.set_index('Item Number', inplace = True)
breakout.set_index('Item Number', inplace = True)
rates_umbra.set_index('item', inplace = True)
rates_ptech.set_index('UPC', inplace = True)
rates_thiele.set_index('UPC', inplace = True)
rates_parsons.set_index('UPC', inplace = True)
demand.set_index('item number', inplace = True)

out_cf.set_index('cf code', inplace = True)
out_cf_orders.set_index('code', inplace = True)
out_fg.set_index('sku', inplace = True)
#out_packline.set_index('line', inplace = True)
#out_bins.set_index('bin', inplace = True)

groups_breakout = breakout.groupby(breakout.index)

In [4]:
class FG_order(object):
       
    def __init__(self, code, val):
        self.code = code
        self.week = val[0]
        self.dem_bags = val[1]
        self.att_bags = val[2]
        self.dem_tons = val[3]
        self.att_tons = val[4]
        self.pline = val[5]
        self.pack_start = val[6]
        self.pack_end = val[7]
        self.prod_hours = val[8]
        
    def not_finished(self, total_demand):
        return total_demand/2000 > self.att_tons;
class Sku(object):
    
    def  __init__(self, code):
        self.code = code
        self.demands = (demand.loc[self.code, demand.columns[3:27].values]).values / 2000
        self.fg_orders = []
        aux_breakout = groups_breakout.get_group(self.code)
        aux_cf = aux_breakout['Component Formula Number'].values
        aux_shrk = aux_breakout['Shrinkage'].values
        aux_percentage = aux_breakout['Blend Percentage'].values
        self.cfs = dict(zip(aux_cf,aux_percentage))
        self.shrinkage = dict(zip(aux_cf,aux_shrk))
        self.weight = aux_breakout['weight'].values[0]
        aux_packname = []
        aux_packrate = []
        if self.code in rates_umbra.index:
            aux_packname.append('umbra')
            aux_packrate.append(rates_umbra.loc[self.code, 'kg/s'])
        if self.code in rates_parsons.index:
            aux_packname.append('parsons')
            aux_packrate.append(rates_parsons.loc[self.code, 'kg/s'])
        if self.code in rates_ptech.index:
            aux_packname.append('ptech')
            aux_packrate.append(rates_ptech.loc[self.code, 'kg/s'])
        if self.code in rates_thiele.index:
            aux_packname.append('thiele')
            aux_packrate.append(rates_thiele.loc[self.code, 'kg/s'])
        self.rate_packlines = dict(zip(aux_packname,aux_packrate))
        

def fill_orders(sku,orders_df):
    sku.fg_orders = [FG_order(row, val) for row,val in orders_df.get_group(sku.code).iterrows()]
    
def fill_skus():
    skus = {}
    orders_df = out_fg.groupby('sku')
    for code in demand.index:
        if code in breakout.index:
            skus[code] = Sku(code)
            if code in orders_df.groups.keys():
                fill_orders(skus[code],orders_df)
        else:
            print(code + " is not in Breakout")
    return skus

big_skus = fill_skus()

WLDKN310111 is not in Breakout
WLDKN310124 is not in Breakout
WLDKN310404 is not in Breakout
WLDKN320111 is not in Breakout
WLDKN320124 is not in Breakout
WLDKN320404 is not in Breakout
NCDKN390150 is not in Breakout
NCDKN440150 is not in Breakout
CHDKN120115 is not in Breakout
CHDKN120130 is not in Breakout
CHDKN120140 is not in Breakout
CHDKN130115 is not in Breakout
CHDKN130130 is not in Breakout
CHDKN130140 is not in Breakout
CHDKN140128 is not in Breakout
CHDKN150128 is not in Breakout
CHDKN160128 is not in Breakout


In [9]:
def give_skus_with_demand(skus):
    skus_dem ={}
    for scode in skus:
        pass
    pass
def give_skus_not_finished(skus, weeks = [1], min_weight = 0, max_weight = 100, min_cf_members = 1, max_cf_members = 5):
    not_finished = {}
    for scode in filter(lambda x: skus[x].demands.sum() > 0 and 
                        ( min_weight <= skus[x].weight and skus[x].weight <= max_weight) and
                        (min_cf_members <= len(skus[x].cfs) and  len(skus[x].cfs) <= max_cf_members), skus):
        
        fgs_not_finished = []
        total_demand = skus[scode].demands.sum()
        att_in_weeks = [fg.att_tons for fg in skus[scode].fg_orders if fg.week in weeks]
        att = sum(att_in_weeks)
        if att < total_demand:
            not_finished[scode] =att
    return not_finished


In [30]:
def give_overall_info(skus, weeks):
    #Take into consideration that what we didnt produced may differ because we sometimes overDo some skus
    demand = 0
    prod = 0
    for scode in big_skus:
        demand += big_skus[scode].demands.sum()
        prod += sum([fg.att_tons for fg in big_skus[scode].fg_orders if fg.week in [1]])
    print("demand:",'{:.2f}'.format(demand),"produced amount:" '{:.2f}'.format(prod),"left to produced:" '{:.2f}'.format(demand - prod))

In [24]:
def give_not_produced_info(skus, weeks, threshold):
    print("Calculating amount not produced for weeks:", weeks)
    print("With a threshold of:", threshold)
    print("=====================")
    not_f = give_skus_not_finished(skus, weeks)
    not_prod = 0
    for scode in not_f:
        if big_skus[scode].demands.sum() - not_f[scode] > threshold:
            print(scode)
            print('\t# component: ', len(big_skus[scode].cfs))
            print('\tweight: ', big_skus[scode].weight)
            print('\tdemand: ', '{:.2f}'.format(big_skus[scode].demands.sum()), ' attained: ', '{:.2f}'.format(not_f[scode]), 'not produced: ', '{:.2f}'.format(big_skus[scode].demands.sum() - not_f[scode]))
            not_prod += big_skus[scode].demands.sum() - not_f[scode]
            print("=====================")

    print("amount not produced:",'{:.2f}'.format(not_prod))

In [31]:
give_not_produced_info(big_skus, [1], 1)
give_overall_info(big_skus, [1])

Calculating amount not produced for weeks: [1]
With a threshold of: 1
PSDKN060130
	# component:  1
	weight:  30.0
	demand:  165.00  attained:  160.80 not produced:  4.20
HBDKN390406
	# component:  1
	weight:  6.0
	demand:  50.00  attained:  45.60 not produced:  4.40
HBDKN430406
	# component:  1
	weight:  6.0
	demand:  50.00  attained:  41.37 not produced:  8.63
WLCKN010307
	# component:  1
	weight:  7.0
	demand:  35.00  attained:  24.08 not produced:  10.92
ALDKN460404
	# component:  5
	weight:  3.5
	demand:  25.00  attained:  0.00 not produced:  25.00
HBDKN370601
	# component:  1
	weight:  1.0
	demand:  12.50  attained:  0.00 not produced:  12.50
ALDKN410406
	# component:  1
	weight:  6.0
	demand:  12.50  attained:  4.83 not produced:  7.67
HBDKN430601
	# component:  1
	weight:  1.0
	demand:  10.00  attained:  0.00 not produced:  10.00
HBDKN400601
	# component:  1
	weight:  1.0
	demand:  5.00  attained:  0.00 not produced:  5.00
HBDKN390601
	# component:  1
	weight:  1.0
	demand:  4.7